<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Text generation using a RNN with eager execution

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
# Download the Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
# Read the data
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Create training examples and targets

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
F
i
r
s
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in dataset.take(1):
  print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print('Step{:4d}'.format(i))
  print(' input: {} ({:s})'.format(input_idx, repr(idx2char[input_idx])))
  print(' expected output: {} ({:s})'.format(target_idx, repr(idx2char[target_idx])))

Step   0
 input: 18 ('F')
 expected output: 47 ('i')
Step   1
 input: 47 ('i')
 expected output: 56 ('r')
Step   2
 input: 56 ('r')
 expected output: 57 ('s')
Step   3
 input: 57 ('s')
 expected output: 58 ('t')
Step   4
 input: 58 ('t')
 expected output: 1 (' ')


In [0]:
# Create training batches
# Batch size
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch // BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      rnn(rnn_units,
          return_sequences=True,
          recurrent_initializer='glorot_uniform',
          stateful=True),
      tf.keras.layers.Dense(vocab_size)
  ])
  
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size = BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Try it for the 1st example in the batch
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [0]:
sampled_indices

array([ 3,  6, 21,  4,  2, 38, 30, 24, 55, 27, 37, 20, 57, 32, 55, 35, 29,
       19, 22, 14, 52, 54, 23, 55, 37, 57, 58,  3,  6, 54, 34, 25, 10, 34,
       55, 24, 12, 49, 21, 51, 45, 55, 35,  6, 17, 20, 25,  8,  4,  2, 61,
       48,  1, 23,  6, 30,  9, 10, 43, 18, 27, 54, 28, 26, 44, 13, 21, 60,
       30,  9, 35, 54, 17, 31, 33, 11, 47, 12, 22, 14, 18, 46, 44, 31, 46,
       48, 32, 46, 47, 10, 46, 56,  2, 36, 34, 33,  7, 30, 29, 28])

In [0]:
# Decode
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 ",\n'Thus saith the duke, thus hath the duke inferr'd;'\nBut nothing spake in warrant from himself.\nWhe"

Next Char Predictions: 
 '$,I&!ZRLqOYHsTqWQGJBnpKqYst$,pVM:VqL?kImgqW,EHM.&!wj K,R3:eFOpPNfAIvR3WpESU;i?JBFhfShjThi:hr!XVU-RQP'


In [0]:
# Train the model
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173156


In [0]:
model.compile(
  optimizer = tf.train.AdamOptimizer(),
  loss = loss)

In [0]:
# Configure checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the ckp file
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath = checkpoint_prefix,
  save_weights_only = True)

In [0]:
EPOCHS = 30

In [0]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/30
173/174 [============================>.] - ETA: 0s - loss: 2.6856WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
174/174 [==============================] - 27s 157ms/step - loss: 2.6824
Epoch 2/30
174/174 [==============================] - 25s 143ms/step - loss: 1.9332
Epoch 3/30
174/174 [==============================] - 25s 144ms/step - loss: 1.6708
Epoch 4/30
174/174 [==============================] - 25s 143ms/step - loss: 1.5292
Epoch 5/30
174/174 [==============================] - 25s 143ms/step - loss: 1.4457
Epoch 6/30
174/174 [==============================] - 25s 144ms/step - loss: 1.3894
Epoch 7/30
174/174 [==============================] - 

In [0]:
# Restore the latest checkpoint
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"JOHN: "))

JOHN: I have print
like to ins, now fear up justice:
These eyes that he gives his disdain I will practise
this word. Thursday war.
Here in this poor fair without disposition.

FLORIZEL:
What, would he swear,
Deep as in a name, shook; what
Master lives: when the senate has gatest
To rave the instruct them on.

JULIET:
Good even too, your parties,--as the east against the Tower:
My Lady Boltague, I'll make it my mead more. or I will have King Henry's art,
Thou didst precedent when he wailing eyes
O court, for this great loss,
That Harry Duke of Norfolk, Thords, when it is defence!
Now are you of the parliament-house!

MENENIUS:
Sir Richard! the nature of this haste.

FRIAR LAURENCE:
Who was the lark, you that wit it. Will't the French.
I talk of men, thy father gave the prisoner. First, would have held him care.

ROMEO:
A hunbraved and ebbraring life in Padua
Of March's window name; I'll plant Patiencl;
I not receive the harmless shades of night
I tell you home to shade, blame that the e

In [0]:
# Customized Training
model = build_model(
  vocab_size = len(vocab),
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size = BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

In [0]:
# Training step
EPOCHS = 1

for epoch in range(EPOCHS):
  start = time.time()
  
  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()
  
  for (batch_n, (inp, target)) in enumerate(dataset):
    with tf.GradientTape() as tape:
      # feeding the hidden state back into the model
      # This is the interesting step
      predictions = model(inp)
      loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {:4f}'
      print(template.format(epoch+1, batch_n, loss))
      
  # saving (ckpt) the model every 5 epochs
  if (epoch+1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch = epoch))
    
  print('Epoch {} Loss {:4f}'.format(epoch+1, loss))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Instructions for updating:
Use tf.cast instead.
Epoch 1 Batch 0 Loss 4.173581
Epoch 1 Batch 100 Loss 2.328911
Epoch 1 Loss 2.131164
Time taken for 1 epoch 26.56835627555847 sec

